In [14]:
(def input "<x=3, y=15, z=8>
<x=5, y=-1, z=-2>
<x=-10, y=8, z=2>
<x=8, y=4, z=-5>")

(defrecord Moon [p v])

(def moons 
    [
      {:p [3 15 8] :v [0 0 0]}      
      {:p [5 -1 -2] :v [0 0 0]}      
      {:p [-10 8 2] :v [0 0 0]}      
      {:p [8 4 -5] :v [0 0 0]}                
            
    ])

(def rmoons 
    [
      (Moon. [3 15 8] [0 0 0])     
      (Moon. [5 -1 -2]  [0 0 0])  
      (Moon. [-10 8 2]  [0 0 0])    
      (Moon. [8 4 -5]  [0 0 0])                
            
    ])


(def testinput "<x=-1, y=0, z=2>
<x=2, y=-10, z=-7>
<x=4, y=-8, z=8>
<x=3, y=5, z=-1>
")

(def testmoons 
    [
      {:p [-1 0 2] :v [0 0 0]}      
      {:p [2 -10 -7] :v [0 0 0]}      
      {:p [4 -8 8] :v [0 0 0]}      
      {:p [3 5 -1] :v [0 0 0]}                
            
    ])

(def test2 "<x=-8, y=-10, z=0>
<x=5, y=5, z=10>
<x=2, y=-7, z=3>
<x=9, y=-8, z=-3>
")

(def test2moons 
    [
      {:p [-8 -10 0] :v [0 0 0]}      
      {:p [5 5 10] :v [0 0 0]}      
      {:p [2 -7 3] :v [0 0 0]}      
      {:p [9 -8 -3] :v [0 0 0]}                
            
    ])


#'user/test2moons

In [15]:
(defn dv-ax [[c1 c2]]
    (cond
        (< c1 c2) 1
        (= c1 c2) 0
        (> c1 c2) -1))


(defn dv [p1 p2]
    (map
     #(dv-ax %)
    
    (map vector p1 p2 )
    ))


(dv [6 2 5] [6 3 2])


(0 1 -1)

In [16]:
(defn addp [p1 p2]
    (mapv #(+ (first %) (last %)) (map vector p1 p2)
    ))

(map + [1 1 5] [-1 3 2])



(0 4 7)

In [17]:
(defn newv [moon all] 
    (let 
        [
         dvs (mapv #(dv (:p moon) (:p %)) all)
         ]
        (reduce  #(mapv + %1 %2 ) (:v moon) dvs)
))

#'user/newv

In [18]:
(defn dograv [all]
    (mapv #(assoc % :v (newv % all)) all)
         )
         
(defn dopos [all]
    (mapv 
     #(assoc % :p (addp (:p %) (:v %)) )
     all
    ))

#'user/dopos

In [19]:
(defn step [all]

(dopos (dograv all)))
    

#'user/step

In [20]:
(defn step2 [[n all]]

[(inc n) (dopos (dograv all))])
    

#'user/step2

In [21]:
(first (drop 1000 (iterate step rmoons)))




[#user.Moon{:p [-56 -6 17], :v [-17 -12 10]} #user.Moon{:p [56 -2 31], :v [2 8 3]} #user.Moon{:p [-14 -42 18], :v [15 0 -14]} #user.Moon{:p [20 76 -63], :v [0 4 1]}]

In [22]:
(defn energy [p]
    (reduce #(+ %1 (Math/abs %2)) 0 p))


#'user/energy

In [23]:
(apply + (map #(*
        (energy (first %))
       (energy (last %))
                ) (map vals
(first (drop 1000 (iterate step moons)))
 )))

7179

In [24]:
(defn get-axis [m n]
    ;;(println m)
    (let [
          pa ((:p m) n)
          va ((:v m) n)
          
          ]
        [pa va]
        )
    )
(defn full-axis [all n]
    (map #(get-axis % n) all))

(set! *print-length* 20)

(defn find-period [all n c axes]
    (let [axs (full-axis  all n)
          idx (axes axs)
          ;;_ (println axes)
          ;;_ (println ax)
    
          ]
    (if idx
        (do (println axs)
            (- c idx))
        (recur (step all) n (inc c) (assoc axes axs c)))
    ))

(defn periods [all] 
    (for [n (range 3)]
    
(find-period all n 0 {}) ))
        

(defn periods2 [all]
    (for [n (range 3)
        :let [xinit (full-axis all n)]]
  (first (first (filter
    #(= xinit (full-axis  (last %) n))
    (drop 1 (iterate step2 [0 all])))
    ))))

#'user/periods2

In [25]:
(def prime-numbers
  ((fn f [x]
     (cons x
       (lazy-seq
         (f (first
              (drop-while
                (fn [n]
                  (some #(zero? (mod n %))
                    (take-while #(<= (* % %) n) prime-numbers)))
                (iterate inc (inc x))))))))
   2))

(defn factorize [n]
  ((fn f [n [h & r :as ps]]
     (cond (< n 2) '()
       (zero? (mod n h)) (cons h (lazy-seq (f (quot n h) ps)))
       :else (recur n r)))
   n prime-numbers))

#'user/factorize

In [26]:
(def ps (time (doall (periods2 rmoons))))


"Elapsed time: 15958.0266 msecs"


#'user/ps

In [64]:
(set! *warn-on-reflection* true)
(time (doall (periods2 rmoons)))


"Elapsed time: 25352.801828 msecs"


(231614 144624 102356)

In [67]:

(defn smallest-period [ps] 
      (apply * 1N (mapcat 
   #(repeat (last %) (first %))

(apply merge-with max

(map (comp frequencies factorize) ps)))))

        
   (smallest-period ps)

428576638953552N

In [51]:
(time (dotimes [n 100000] (full-axis rmoons 0)))



"Elapsed time: 21.957653 msecs"


nil

In [27]:
(def input "<x=3, y=15, z=8>
<x=5, y=-1, z=-2>
<x=-10, y=8, z=2>
<x=8, y=4, z=-5>")

(defrecord Moon [p v])

(def moons 
    [
      {:p [3 15 8] :v [0 0 0]}      
      {:p [5 -1 -2] :v [0 0 0]}      
      {:p [-10 8 2] :v [0 0 0]}      
      {:p [8 4 -5] :v [0 0 0]}                
            
    ])

(def rmoons 
    [
      (Moon. [3 15 8] [0 0 0])     
      (Moon. [5 -1 -2]  [0 0 0])  
      (Moon. [-10 8 2]  [0 0 0])    
      (Moon. [8 4 -5]  [0 0 0])                
            
    ])


(def testinput "<x=-1, y=0, z=2>
<x=2, y=-10, z=-7>
<x=4, y=-8, z=8>
<x=3, y=5, z=-1>
")

(def testmoons 
    [
      {:p [-1 0 2] :v [0 0 0]}      
      {:p [2 -10 -7] :v [0 0 0]}      
      {:p [4 -8 8] :v [0 0 0]}      
      {:p [3 5 -1] :v [0 0 0]}                
            
    ])

(def test2 "<x=-8, y=-10, z=0>
<x=5, y=5, z=10>
<x=2, y=-7, z=3>
<x=9, y=-8, z=-3>
")

(def test2moons 
    [
      {:p [-8 -10 0] :v [0 0 0]}      
      {:p [5 5 10] :v [0 0 0]}      
      {:p [2 -7 3] :v [0 0 0]}      
      {:p [9 -8 -3] :v [0 0 0]}                
            
    ])


#'user/test2moons

In [28]:
(p2)

Syntax error compiling at (REPL:1:1).
Unable to resolve symbol: p2 in this context


class clojure.lang.Compiler$CompilerException: 

In [29]:
(ns proj.p12
  (:require [clojure.string :refer [split split-lines trim-newline]]
            [clojure.math.combinatorics :as combs :refer [permutations combinations]]
            [proj.utils :refer [lcm]]))

(def ip (->> "p12.txt"
             slurp
             split-lines
             (map #(re-matcher #"-?\d+" %))
             (mapv (fn [mat]
                     (vec (for [_ (range 3)]
                            (read-string (re-find mat))))))))

(defn abs [a]
  (Math/abs a))

(defn potential [p]
  (->> p
       (map abs)
       (reduce +)))

(defn kinetic [v]
  (->> v
       (map abs)
       (reduce +)))

(defn calc-energy [ps vs]
  (reduce +
          (map
            #(* (potential %1)
                (kinetic %2))
            ps vs)))

(def ovels (mapv
             (fn [_] (mapv (fn [_] 0) (range 3)))
             (range 4)))

(defn p1 []
  (loop [t 0
         vels ovels
         poss ip]
    (if (= t 1000)
      (calc-energy poss vels)
      (let [new-vels (mapv
                       (fn [i]
                         (let [curr (poss i)]
                           (reduce
                             (fn [v po]
                               (mapv
                                 (fn [vi po pi]
                                   (cond
                                     (< pi po) (inc vi)
                                     (> pi po) (dec vi)
                                     :else vi))
                                 v po curr))
                             (vels i) poss)))
                       (range 4))
            new-poss (mapv
                       (fn [p v]
                         (mapv + p v))
                       poss new-vels)]
        (recur (inc t)
               new-vels
               new-poss)))))

(defn linear-period [ovels oposs]
  (loop [t 0
         vels ovels
         poss oposs]
    (let [new-vels (map
                     (fn [vc pc]
                       (reduce
                         (fn [v po]
                           (cond
                             (< pc po) (inc v)
                             (> pc po) (dec v)
                             :else v))
                         vc poss))
                     vels poss)
          new-poss (map
                     +
                     poss new-vels)]
      (if (and (= new-vels ovels)
               (= new-poss oposs))
        (inc t)
        (recur (inc t)
               new-vels
               new-poss)))))

(defn p2 []
  (apply lcm (for [i (range 3)]
               (let [vis [0 0 0 0]
                     pis (map
                           (fn [p] (p i))
                           ip)]
                 (linear-period vis pis)))))

Execution error (FileNotFoundException) at proj.p12/eval4429$loading (REPL:1).
Could not locate clojure/math/combinatorics__init.class, clojure/math/combinatorics.clj or clojure/math/combinatorics.cljc on classpath.
Execution error (FileNotFoundException) at java.io.FileInputStream/open0 (FileInputStream.java:-2).
p12.txt (The system cannot find the file specified)
Syntax error compiling at (REPL:91:3).
Unable to resolve symbol: lcm in this context


class clojure.lang.Compiler$CompilerException: 